In [1]:
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
from sqlalchemy import inspect
from dotenv import load_dotenv
import os
import pymysql
import pandas as pd

In [2]:
# Cargo las claves del aarchivo .env
load_dotenv() 

# URL de conexion
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "3306")
DB_NAME = os.getenv("DB_NAME")
DATABASE_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [3]:
# creo un engine de SQLAlchemy
engine = create_engine(DATABASE_URL)

# creo la sesion para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base() #declaro 'Base'

In [4]:
# Test de conexion
try:
    connection = engine.connect() #me conecto y consulto algo simple
    print("Conexión exitosa a la base de datos")
except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos: {e}")
finally:
    connection.close()  # cierro la conexión


Conexión exitosa a la base de datos


In [6]:
# Crear la tabla
test_create_table_query = text("""
CREATE TABLE IF NOT EXISTS publishers(
    publisher_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    PRIMARY KEY(publisher_id)
);
""")

with engine.connect() as connection:
    connection.execute(test_create_table_query)
    connection.commit()  # hago el commit para confirmar


In [7]:
create_authors_tables = text("""
CREATE TABLE IF NOT EXISTS authors(
    author_id INT NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    middle_name VARCHAR(50) NULL,
    last_name VARCHAR(100) NULL,
    PRIMARY KEY(author_id)
);
""")
with engine.connect() as connection:
    connection.execute(create_authors_tables)
    connection.commit()  # hago el commit para confirmar

In [8]:
create_books_table = text("""
CREATE TABLE IF NOT EXISTS books(
    book_id INT NOT NULL,
    title VARCHAR(255) NOT NULL,
    total_pages INT NULL,
    rating DECIMAL(4, 2) NULL,
    isbn VARCHAR(13) NULL,
    published_date DATE,
    publisher_id INT NULL,
    PRIMARY KEY(book_id),
    CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id)
);
""")
create_book_authors_table = text("""
CREATE TABLE IF NOT EXISTS book_authors (
    book_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY(book_id, author_id),
    CONSTRAINT fk_book FOREIGN KEY(book_id) REFERENCES books(book_id) ON DELETE CASCADE,
    CONSTRAINT fk_author FOREIGN KEY(author_id) REFERENCES authors(author_id) ON DELETE CASCADE
);
""")

with engine.connect() as connection:
    connection.execute(create_books_table)
    connection.execute(create_book_authors_table)
    connection.commit()  # hago el commit para confirmar

In [9]:
# Crear una conexión a la base de datos
with engine.connect() as connection:
    # Crear un inspector para la base de datos
    inspector = inspect(connection)
    
    # Obtener la lista de tablas
    tables = inspector.get_table_names()
    
    # Mostrar las tablas
    print("Tablas en la base de datos:")
    for table in tables:
        print(table)

Tablas en la base de datos:
authors
book_authors
books
publishers


In [23]:
table_of_query = 'publishers'
query = text(f'SELECT * FROM {table_of_query};')

with engine.connect() as connection:
    result = connection.execute(query) 
    for i in result : 
        print(i)
    else:
        print("No se encontraron resultados")


No se encontraron resultados


In [38]:

sql_directory = 'C:/Users/Agustín/Desktop/4Geeks/Clases/11. Intro to SQL/connecting-to-a-sql-database-project-tutorial/src/sql'

with open('C:/Users/Agustín/Desktop/4Geeks/Clases/11. Intro to SQL/connecting-to-a-sql-database-project-tutorial/src/sql/insert.sql', 'r') as sql_file:
    sql_commands = sql_file.read()
    with engine.connect() as connection:
        transaction = connection.begin()  # Comienza una transacción
        try: 
            for command in sql_commands.split(';'):
                command = command.strip()
                if command:
                    connection.execute(text(command))
            transaction.commit()
        except Exception as e:
            transaction.rollback()
            print(f"Error: {e}")
        connection.commit()  # Hacer el commit para confirmar las inserciones




In [49]:
table_of_query = 'publishers'
query = text(f'SELECT * FROM {table_of_query};')

with engine.connect() as connection:
    df_result = pd.read_sql(query,connection)
    if df_result.empty:
        print("No se encontraron resultados")
    else :
        print(df_result)

   publisher_id             name
0             1   O Reilly Media
1             2     A Book Apart
2             3       A K PETERS
3             4   Academic Press
4             5   Addison Wesley
5             6  Albert&Sweigart
6             7  Alfred A. Knopf
